## Projet RI

### Prédiction des caractéristiques physiques du carbone noir à l'aide de variables Lidar

Ci-dessous est codée une fonction qui permet de renseigner les caractéristiques physiques d'une particule de carbone noir. Ces caractéristiques sont obtenues à partir de 4 élèments : la valeur de son Color Ration associé, de son Backscattering Angle Effect, de sa dimension fractale et pourcentage de coating.
Pour obtenir ces informations, différents modèles de Machine Learning ont été entrainés au préalable et seulement les modèles donnant les meilleurs résultats ont été sélectionnés. Dans notre cas les meilleurs résultats ont été obtenus :
- pour 'primary_particle_size (nm)': avec 'GBR',
- pour 'vol_equi_radius_outer (nm)': avec 'KRR',
- pour 'vol_equi_radius_inner (nm)': avec 'KRR',
- pour 'equi_mobility_dia (nm)': avec 'KRR',
- pour 'mass_bc (g)': avec 'KRR'

Pour utiliser la fonction 'Predict_parameters', veuillez renseigner les éléments suivants : 
- 'cr' : Color Ratio (float)
- 'bae' : Backscattering Angle Effect (float)
- 'fractal_dimension' : Dimension Fractale (float)
- 'fraction_of_coating' : Pourcentage de Coating (integer, %)


Si vous n'avez pas la valeur de CR ou BAE, entrez 0 à la place du paramètre.

**Indication :**

Avant tout chose, assurez vous de télécharger l'excel 'df_cleaned' ainsi que les poids 

In [48]:
import numpy as np
from joblib import load
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [59]:
df = pd.read_excel('./df_cleaned.xlsx')

X = df.iloc[:, 2:23]  # données particules
Y = df.iloc[:,23:31]  # données optiques
L_all = df.iloc[:, [0, 1] + list(range(31, df.shape[1]))]

# Sans BAE 
L_CR = L_all.iloc[:,0:3]

# Sans CR 
L_BAE = L_all.drop(L_all.columns[2], axis=1)

X_test_all=L_all.reset_index(drop=True)
X_test_CR=L_CR.reset_index(drop=True)
X_test_BAE =L_BAE.reset_index(drop=True)

scaler_all = StandardScaler()
scaler_CR = StandardScaler()
scaler_BAE  = StandardScaler()

scaler_all.fit(X_test_all)   #pt.transform(X_test)
scaler_CR.fit(X_test_CR)
scaler_BAE.fit(X_test_BAE)

StandardScaler()

In [60]:
# Si vous n'avez pas les valeurs de CR ou BAE rentrez 0 comme argument 
# Pensez à changer l'argument model_dir en fonction de la localisation des poids sur votre PC 
def Predict_parameters(cr, bae, fractal_dimension, fraction_of_coating):

    # Variable à prédire
    var_target = {
         'primary_particle_size (nm)': 'GBR',
         'vol_equi_radius_outer (nm)': 'KRR',
         'vol_equi_radius_inner (nm)': 'KRR',
         'equi_mobility_dia (nm)': 'KRR',
         'mass_bc (g)': 'KRR'}
     # Répertoire des modèles sauvegardés
    if cr == 0 : 
        model_dir = "./Best_models/L_X_BAE_best_models"
        input_data = np.array([fractal_dimension, fraction_of_coating,bae]).reshape(1, -1)
        input_data = scaler_BAE.transform(input_data)
        print(input_data)
        r2_values = {
            'R2 primary_particle_size (nm)': 1 ,
            'R2 vol_equi_radius_outer (nm)': 0.917,
            'R2 vol_equi_radius_inner (nm)': 0.925,
            'R2 equi_mobility_dia (nm)': 0.915,
            'R2 mass_bc (g)': 0.85
        }
    elif bae == 0 :
        model_dir = "./Best_models/L_X_CR_best_models"
        input_data = np.array([fractal_dimension, fraction_of_coating,cr]).reshape(1, -1)
        input_data = scaler_CR.transform(input_data)
        print(input_data)
        r2_values = {
            'R2 primary_particle_size (nm)': 1 ,
            'R2 vol_equi_radius_outer (nm)': 0.916,
            'R2 vol_equi_radius_inner (nm)': 0.924,
            'R2 equi_mobility_dia (nm)': 0.912,
            'R2 mass_bc (g)': 0.53
        }
        
    else : 
        model_dir = "Best_models/L_X_best_models"
        input_data = np.array([fractal_dimension, fraction_of_coating,cr,bae]).reshape(1, -1)
        input_data = scaler_all.transform(input_data)
        print(input_data)
        r2_values = {
            'R2 primary_particle_size (nm)': 1,
            'R2 vol_equi_radius_outer (nm)': 0.916,
            'R2 vol_equi_radius_inner (nm)': 0.924,
            'R2 equi_mobility_dia (nm)': 0.912,
            'R2 mass_bc (g)': 0.54
        }
        
        

    # Données d'entrée pour la prédiction

    predictions = {}
    
    #Pour chaque variable target on effectue la prédiction sur son modèle associé
    for target, model_type in var_target.items():
        
        # Chemin du modèle
        model_path = os.path.join(model_dir, f'{target}_best_model_{model_type}.joblib')
        # Vérification de l'existence du fichier
        if not os.path.exists(model_path):
            raise FileNotFoundError(f"Modèle pour '{target}' introuvable à {model_path}")

        # Chargement du modèle
        model = load(model_path)
        # Prédiction
        predictions[target] = np.abs(model.predict(input_data)[0]) #ces valeurs doivent être positives 

    return predictions,r2_values


### Exemple d'utilisation : 

#### Exemple fictif 

In [66]:
# Si pas de valeurs de CR et BAE mettre 0 à la place 
# Les valeurs de R2 nous donne une indication sur l'accuracy 

#'cr', n'a pas de valeur 
#'bae'=-3.9
#'fractal_dimension'=1.5
#'fraction_of_coating'=0%

Predict_parameters(0, -3.9 , 1.5, 1)

[[-1.41299233 -1.1845503  -0.8756944 ]]


C:\Users\misse\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


({'primary_particle_size (nm)': 15.100000011512149,
  'vol_equi_radius_outer (nm)': 22.58890496251354,
  'vol_equi_radius_inner (nm)': 22.43995978986586,
  'equi_mobility_dia (nm)': 45.439233550627776,
  'mass_bc (g)': 6.342500567279678e-17},
 {'R2 primary_particle_size (nm)': 1,
  'R2 vol_equi_radius_outer (nm)': 0.917,
  'R2 vol_equi_radius_inner (nm)': 0.925,
  'R2 equi_mobility_dia (nm)': 0.915,
  'R2 mass_bc (g)': 0.85})

In [67]:
#'cr'=0.8 
#'bae' n'a pas de valeur 
#'fractal_dimension'=1.8
#'fraction_of_coating'=20%

Predict_parameters(0.6, 0 , 1.5, 15)

[[-1.41299233 -0.62179932 -1.02378737]]


C:\Users\misse\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


({'primary_particle_size (nm)': 15.800000003353945,
  'vol_equi_radius_outer (nm)': 26.722569126737604,
  'vol_equi_radius_inner (nm)': 25.315966770429988,
  'equi_mobility_dia (nm)': 56.76044587271012,
  'mass_bc (g)': 7.287019822337014e-17},
 {'R2 primary_particle_size (nm)': 1,
  'R2 vol_equi_radius_outer (nm)': 0.916,
  'R2 vol_equi_radius_inner (nm)': 0.924,
  'R2 equi_mobility_dia (nm)': 0.912,
  'R2 mass_bc (g)': 0.53})

In [68]:
#'cr'=0.8 
#'bae'=-2.8
#'fractal_dimension'=1.8
#'fraction_of_coating'=20%

Predict_parameters(0.8, -2.8 , 1.8, 20)

[[-0.34784933 -0.42081683  1.65934161  0.38618727]]


C:\Users\misse\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


({'primary_particle_size (nm)': 16.20000000091781,
  'vol_equi_radius_outer (nm)': 0.0011519513986543144,
  'vol_equi_radius_inner (nm)': 0.0010905446508611347,
  'equi_mobility_dia (nm)': 0.007044880868001489,
  'mass_bc (g)': 9.833607505725366e-20},
 {'R2 primary_particle_size (nm)': 1,
  'R2 vol_equi_radius_outer (nm)': 0.916,
  'R2 vol_equi_radius_inner (nm)': 0.924,
  'R2 equi_mobility_dia (nm)': 0.912,
  'R2 mass_bc (g)': 0.54})

#### Exemple avec valeurs du jeu de données 

In [69]:
cr=0.66628753296408
bae= -3.20854360536531
fractal_dimension=1.7
fraction_of_coating =40 #%

Predict_parameters(cr, bae ,fractal_dimension , fraction_of_coating)

# On attent 
# primary_particle_size (nm) = 17,8 
#'vol_equi_radius_outer (nm)' = 165
# vol_equi_radius_inner (nm) = 139 
# equi_mobility_dia (nm) = 855
# mass_bc (g)' = 1,7 e-14

[[-0.70289699  0.38311314 -0.13449737 -0.08247972]]


C:\Users\misse\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


({'primary_particle_size (nm)': 17.799999996956316,
  'vol_equi_radius_outer (nm)': 157.12902535048357,
  'vol_equi_radius_inner (nm)': 132.31977926855507,
  'equi_mobility_dia (nm)': 795.368182419443,
  'mass_bc (g)': 1.3100853246883575e-14},
 {'R2 primary_particle_size (nm)': 1,
  'R2 vol_equi_radius_outer (nm)': 0.916,
  'R2 vol_equi_radius_inner (nm)': 0.924,
  'R2 equi_mobility_dia (nm)': 0.912,
  'R2 mass_bc (g)': 0.54})

In [70]:
cr=0.697442625
bae= -2.847423437
fractal_dimension=1.5
fraction_of_coating =15 #%

Predict_parameters(cr, bae ,fractal_dimension , fraction_of_coating)

# On attent 
# primary_particle_size (nm) = 15,8
#'vol_equi_radius_outer (nm)' = 116,4153954
# vol_equi_radius_inner (nm) = 110,520945 
# equi_mobility_dia (nm) = 533,007323
# mass_bc (g)' = 8,48E-15

[[-1.41299233 -0.62179932  0.28346828  0.33178476]]


C:\Users\misse\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


({'primary_particle_size (nm)': 15.800000003353947,
  'vol_equi_radius_outer (nm)': 116.13884652565329,
  'vol_equi_radius_inner (nm)': 110.11392705223531,
  'equi_mobility_dia (nm)': 533.1303527868462,
  'mass_bc (g)': 8.921608373220716e-15},
 {'R2 primary_particle_size (nm)': 1,
  'R2 vol_equi_radius_outer (nm)': 0.916,
  'R2 vol_equi_radius_inner (nm)': 0.924,
  'R2 equi_mobility_dia (nm)': 0.912,
  'R2 mass_bc (g)': 0.54})

In [71]:
cr=0.606022585
bae= -3.957699995
fractal_dimension=1.7
fraction_of_coating =0 #%

Predict_parameters(cr, bae ,fractal_dimension , fraction_of_coating)

# On attent 
# primary_particle_size (nm) = 16,2
#'vol_equi_radius_outer (nm)' = 12,85794852
# vol_equi_radius_inner (nm) = 11,90550789
# equi_mobility_dia (nm) = 18,07237419
# mass_bc (g)' =1,06E-17

[[-0.70289699 -1.2247468  -0.94299051 -0.94188583]]


C:\Users\misse\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


({'primary_particle_size (nm)': 15.000000036308496,
  'vol_equi_radius_outer (nm)': 18.780275056168755,
  'vol_equi_radius_inner (nm)': 18.732138028052567,
  'equi_mobility_dia (nm)': 34.4159296951012,
  'mass_bc (g)': 5.779276916362057e-17},
 {'R2 primary_particle_size (nm)': 1,
  'R2 vol_equi_radius_outer (nm)': 0.916,
  'R2 vol_equi_radius_inner (nm)': 0.924,
  'R2 equi_mobility_dia (nm)': 0.912,
  'R2 mass_bc (g)': 0.54})

In [72]:
cr=0.582127917
bae= -4.275580337
fractal_dimension=1.9
fraction_of_coating =30 #%

Predict_parameters(cr, bae ,fractal_dimension , fraction_of_coating)

# On attent 
# primary_particle_size (nm) =16,9
#'vol_equi_radius_outer (nm)' = 35,15341661
# vol_equi_radius_inner (nm) = 31,20125735
# equi_mobility_dia (nm) = 82,33336731
# mass_bc (g)' = 1,91E-16

[[ 0.00719834 -0.01885185 -1.26355289 -1.30654708]]


C:\Users\misse\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


({'primary_particle_size (nm)': 16.899999998433167,
  'vol_equi_radius_outer (nm)': 33.685307025364324,
  'vol_equi_radius_inner (nm)': 29.868461018088375,
  'equi_mobility_dia (nm)': 77.85654325335973,
  'mass_bc (g)': 1.4963798619463382e-16},
 {'R2 primary_particle_size (nm)': 1,
  'R2 vol_equi_radius_outer (nm)': 0.916,
  'R2 vol_equi_radius_inner (nm)': 0.924,
  'R2 equi_mobility_dia (nm)': 0.912,
  'R2 mass_bc (g)': 0.54})